In [ ]:
from google.colab import files
uploaded=files.upload()

Saving mobilenetv2_cifar10.keras to mobilenetv2_cifar10 (2).keras


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical


In [ ]:
import tensorflow as tf
import numpy as np

# Load CIFAR-10 for testing
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# One-hot encode labels
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

# ImageNet normalization constants
imagenet_mean = tf.constant([0.485, 0.456, 0.406], dtype=tf.float32)
imagenet_std = tf.constant([0.229, 0.224, 0.225], dtype=tf.float32)

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    image = (image - imagenet_mean) / imagenet_std
    return image, label

# Create test dataset pipeline
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test_cat)) \
                         .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE) \
                         .batch(64) \
                         .prefetch(tf.data.AUTOTUNE)


In [ ]:
# Load the saved model
model = tf.keras.models.load_model("mobilenetv2_cifar10.keras")


TFLITE CONVERSION

In [ ]:
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

Saved artifact at '/tmp/tmp9vuz2l3o'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134841425554576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134841425556496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134841425556304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134841425556688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134841425555920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134841425556112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134841425558224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134841425555152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134841425557264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134841425558800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134841425559

In [ ]:
len(tflite_model)

8915484

In [ ]:
# Save the TFLite model to a file
with open("mobilenetv2_cifar10.tflite", "wb") as f:
    f.write(tflite_model)

# Download the file to your local computer
from google.colab import files
files.download("mobilenetv2_cifar10.tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Post Training *Quantization*

In [ ]:
# post training quantized model
converter=tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations=[tf.lite.Optimize.DEFAULT]
tflite_quantized_model=converter.convert()

Saved artifact at '/tmp/tmpxdyqc8wl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  135472319966928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135472319967888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135472319966160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135472319967120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135472319967312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135472319966352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135472319966736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135472319968464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135472319955216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135472319968080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135472319966

In [ ]:
# Save post training quantized TFLite model to file
with open("mobilenetv2_cifar10_quantized.tflite", "wb") as f:
    f.write(tflite_quantized_model)

# Download it to your local computer
from google.colab import files
files.download("mobilenetv2_cifar10_quantized.tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PTQ accuracy

In [ ]:
import tensorflow as tf
import numpy as np

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="mobilenetv2_cifar10_quantized.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input shape:", input_details[0]['shape'], "dtype:", input_details[0]['dtype'])


Input shape: [  1 224 224   3] dtype: <class 'numpy.float32'>


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [ ]:
# Get quantization parameters for input
input_scale, input_zero_point = input_details[0]['quantization']

def preprocess_tflite(image):
    # Resize to 224x224
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    image = (image - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]  # ImageNet norm

    # Quantize if required
    if input_details[0]['dtype'] == np.uint8 or input_details[0]['dtype'] == np.int8:
        image = image / input_scale + input_zero_point
        image = tf.clip_by_value(image, 0, 255)  # for uint8 range
        image = tf.cast(image, input_details[0]['dtype'])

    return image


In [ ]:
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_test = y_test.flatten()  # convert to shape (N,)

correct = 0
total = 0

for i in range(len(x_test)):
    img = preprocess_tflite(x_test[i])
    img = np.expand_dims(img, axis=0)  # batch size 1

    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])[0]

    pred_label = np.argmax(output)
    if pred_label == y_test[i]:
        correct += 1
    total += 1

accuracy = correct / total
print("Quantized model accuracy: {:.2f}%".format(accuracy * 100))


Quantized model accuracy: 87.18%
